# Pràctica 2: Recomanador Heurístic

Nom dels alumnes del grup: Andres Rio i Aleix Falgosa


Com fer i estructurar el codi per fer una bona pràctica: 

+ Definir els paràmetres (input) i el retorn (output) de les funcions de forma clara
+ Definir un ordre per defecte dels usuaris/items. Per exemple, si demanen "el film més ben puntuat", ha de ser el que té un 5 i ID més baix.
+ És MOLT important que la funció que calcula la similitud entre les puntuacions en comú de dos usuaris sigui ràpida!
+ Feu unit-tests de Python

## 1. INTRODUCCIÓ

### 1.1. Abans de començar...

**\+ A més a més de les que ja es troben presents en la 1a cel·la i funcions natives de Python, durant la pràctica, només es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple, les funcions tindran [pydoc](https://docs.python.org/3/library/pydoc.html) i allà s'especificarà el paràmetre: `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades.

### 1.2. Dades: puntuacions de pel·licules

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·lícules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/). 

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserID** de l'usuari, amb id's entre 1 i 6040 
- **MovieID** de la pel·licula, amb id's entre 1 i 3952
- **Rating** d'un usuari per una pel·licula, en una escala de 1 (menys) a 5 (més) estrelles.
- **Timestamp** que representa quan aquest usuari va puntuar la pel·licula, representat en segons.

La base de dades original està filtrada de manera que cada usuari té com a mínim 20 puntuacions.

### 1.3. Dades: usuaris



Al fitxer ``users.dat`` hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

Els usuaris han donat la informació voluntariament. Així doncs, la informació d'alguns usuaris pot estar buida.


### 1.4. Dades: pel·lícules



Al fitxer ``movies.dat`` hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes, que estan separats pel símbol "|" i estan seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

Algunes películes poden tenir l'ID malament degut a duplicats accidentals.

Les películes s'han entrat manualment, així que poden existir altres inconsistencies. 

## 2. Exploració de les dades

### 2.1 Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

In [1]:
# executeu aquesta cel·la per baixar les dades d'internet
# al campus virtual hi ha un fitxer que podeu baixar també
import os
if os.path.isfile("/etc/password.txt") == False:
    os.system('wget -nc http://files.grouplens.org/datasets/movielens/ml-1m.zip')
    os.system('unzip ml-1m.zip')

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [5]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [6]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')

### 2.2 Inspecció de les taules

In [7]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
users[-5:]

,user_id,gender,age,occupation,zip
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [9]:
ratings[-5:]

,user_id,movie_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [10]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
ratings.sort_values('movie_id')[:5]

,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [12]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2.3 Exemple: Com extreure informació d'un DataFrame.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**, dades que estan a frames diferents.

El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció.

Reviseu aquests conceptes de pandas: https://pandas.pydata.org/docs/user_guide/merging.html

In [14]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificador d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
29,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller
30,1,2294,4,978824291,F,1,10,48067,Antz (1998),Animation|Children's
31,1,3186,4,978300019,F,1,10,48067,"Girl, Interrupted (1999)",Drama
32,1,1566,4,978824330,F,1,10,48067,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
33,1,588,4,978824268,F,1,10,48067,Aladdin (1992),Animation|Children's|Comedy|Musical
34,1,1907,4,978824330,F,1,10,48067,Mulan (1998),Animation|Children's
35,1,783,4,978824291,F,1,10,48067,"Hunchback of Notre Dame, The (1996)",Animation|Children's|Musical
36,1,1836,5,978300172,F,1,10,48067,"Last Days of Disco, The (1998)",Drama
37,1,1022,5,978300055,F,1,10,48067,Cinderella (1950),Animation|Children's|Musical


In [15]:
data[data['user_id'] == 2]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
53,2,1357,5,978298709,M,56,16,70072,Shine (1996),Drama|Romance
54,2,3068,4,978299000,M,56,16,70072,"Verdict, The (1982)",Drama
55,2,1537,4,978299620,M,56,16,70072,Shall We Dance? (Shall We Dansu?) (1996),Comedy
56,2,647,3,978299351,M,56,16,70072,Courage Under Fire (1996),Drama|War
57,2,2194,4,978299297,M,56,16,70072,"Untouchables, The (1987)",Action|Crime|Drama
...,...,...,...,...,...,...,...,...,...,...
177,2,356,5,978299686,M,56,16,70072,Forrest Gump (1994),Comedy|Romance|War
178,2,1245,2,978299200,M,56,16,70072,Miller's Crossing (1990),Drama
179,2,1246,5,978299418,M,56,16,70072,Dead Poets Society (1989),Drama
180,2,3893,1,978299535,M,56,16,70072,Nurse Betty (2000),Comedy|Thriller


La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes indexades per un enter:

In [16]:
data.iloc[3:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


Els índexs Booleans ens permeten seleccionar una part de la taula que compleix una condició.

In [17]:
# comptem quin tant per cent de ratings estan fets per una dona

print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count())*100, '%')

24.638850480249626 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes.

Reviseu aquests conceptes: 
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table

In [18]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [19]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()``.

Reviseu aquest concepte: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [20]:
ratings_by_title = data.groupby('title').size()
print(ratings_by_title)

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64


Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [21]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [22]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [23]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [24]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [25]:
print(np.nan + 9.0) 

nan


Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [26]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [27]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [28]:
rating_std_by_title = data.groupby('title')['rating'].std()

rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### Important: Temes de rendiment

Fixeu-vos en el comportament de Python en aquests tres exmepls (que tenen el mateix output). Identifiqueu l'origen de les diferències i actueu en conseqüència:

In [29]:
# Aquesta cel·la pot trigar uns segons a executar-se

%timeit data['title'] 
print(type(data['title']))
%timeit data.title 
print(type(data.title))
%timeit data[['title']] 
print(type(data[['title']]))

2.81 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
<class 'pandas.core.series.Series'>
7.14 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
<class 'pandas.core.series.Series'>
7.58 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
<class 'pandas.core.frame.DataFrame'>


## 3. EXERCICIS

### 3.1. EXERCICI A

+ Donada la taula ``data`` tal i com es defineix a continuació, calcula la puntuació mitjana de cada usuari i guarda-la a un ``df`` anomenat ``users_mean_rating``. 

In [30]:
data_folder = 'ml-1m'

In [32]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)

# la vostra solució aquí

users_mean_rating = data.pivot_table(values='rating',aggfunc='mean',index='user_id')
# Amb el rating apliquem la funció de mitjana

users_mean_rating.rename(columns={'rating': 'mean_rating'}, inplace=True) # canviem el nom de la columna
users_mean_rating = users_mean_rating.sort_values(by='mean_rating', ascending=False) # ordenar en funció de la mitjana més alta

users_mean_rating.head(10)

,mean_rating
user_id,
283,4.962963
2339,4.956522
3324,4.904762
3902,4.890909
446,4.843137
447,4.837838
4649,4.818182
4634,4.813725
1131,4.796117


+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable de tipus ``string`` anomenada ``best_movie_rating`` ). 

In [36]:
# la vostra solució aquí

movie_rating = data.groupby('movie_id')['rating'].mean().reset_index() # Calcular la puntuació mitjana por película

best_movie = movie_rating.sort_values(by=['rating', 'movie_id'], ascending=[False, True]).iloc[0] 
# Ordenar per rating (descendent) y després per movie_id (ascendent) y conseguir el seu ID

best_movie_rating = data.loc[data['movie_id'] == best_movie['movie_id'], 'title'].iloc[0]
# para el movie id que sigui igual, agafem el títul

best_movie_rating

'Gate of Heavenly Peace, The (1995)'

+ Mira si hi ha més pel·licules amb la mateixa puntuació de la més ben puntuada.

In [37]:
# la vostra solució aquí

movie_rating = data.groupby(['movie_id', 'title'])['rating'].mean().reset_index() 
# Agrupar per peli ID y título i calcular la puntuació mitjana per pel·lícula

movie_rating = movie_rating.sort_values(by=['rating', 'movie_id'], ascending=[False, True]) 
# Ordenar per puntuació mitjana (descendent) i movie_id (ascendent)

best_movies = movie_rating[movie_rating['rating'] == movie_rating['rating'].max()] # Obtenir les pel·lícules amb la puntuació màxima

# Mostrar les pel·lícules amb la millor puntuació
print(best_movies)

      movie_id                                      title  rating
744        787         Gate of Heavenly Peace, The (1995)     5.0
926        989  Schlafes Bruder (Brother of Sleep) (1995)     5.0
1652      1830                    Follow the Bitch (1998)     5.0
2955      3172                    Ulysses (Ulisse) (1954)     5.0
3010      3233                       Smashing Time (1967)     5.0
3054      3280                           Baby, The (1973)     5.0
3152      3382                     Song of Freedom (1936)     5.0
3367      3607                   One Little Indian (1973)     5.0
3414      3656                               Lured (1947)     5.0
3635      3881                   Bittersweet Motel (2000)     5.0


+ Busca ara aquella pel·lícula, d'entre les que tenen 5 com a puntuació mitjana, que hagi rebut més valoracions i guarda-la a una variable anomenada ``best_movie_rating_maxviews``. Aixi tindrem la pel·licula més ben puntuada per més usuaris. 

In [40]:
# la vostra solució aquí

movie_stats = data.groupby(['movie_id', 'title']).agg(
    rating_mean=('rating', 'mean'),
    rating_count=('rating', 'count')
).reset_index()
# ara apliquem dues funcions, una que fa la mitjana i l'altre que conta la quantitat de valoracions

# Filtrar pel·lícules amb puntuació mitjana de 5
movies_with_max_rating = movie_stats[movie_stats['rating_mean'] == movie_stats['rating_mean'].max()]

# Trobar la pel·lícula amb més valoracions
best_movie_rating_maxviews = movies_with_max_rating.sort_values(by='rating_count', ascending=False).iloc[0]

# Resultat
print(best_movie_rating_maxviews)

movie_id                                       787
title           Gate of Heavenly Peace, The (1995)
rating_mean                                    5.0
rating_count                                     3
Name: 744, dtype: object


### 3.2. EXERCICI B

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.


In [41]:
def top_movie(dataFrame,usr):
    # la vostra solució aquí
    
    valoracions_user = dataFrame[ dataFrame['user_id'] == usr ]  # agrupem tota la info del usuari
    millor_puntuada = valoracions_user[ valoracions_user['rating'] == valoracions_user['rating'].max()]  # filtrar només les que tinguin rating màxim

    la_millor = millor_puntuada.sort_values(by='movie_id').iloc[0] # agafem el ID més baix

    return la_millor['title']
    # la vostra solució aquí

print(top_movie(data,1))

Toy Story (1995)


### 3.3. EXERCICI C

+ Construeix una funció que donat el dataframe ``data`` et retorni un altre dataframe ``df_counts``amb el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes estiguin indexades per `movie_id`, les files per `user_id` i els valors siguin el rating donat.

In [43]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    la valoració que un usuari ha donat a una peli d'un `movie_id`
    
    :param df: DataFrame original 
    :return: DataFrame descrit adalt
    """
    
    # la vostra solució aquí
    return df.pivot( values='rating' , index = 'user_id' , columns = 'movie_id' ) 
    # utilitzem la funció pivot que redistribueix la nostra taula sense afectar el valor del rating

In [44]:
df_counts = build_counts_table(data)
df_counts

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


+ Fés una funció `get_count` que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [46]:
def get_count(df, user_id, movie_id):
    """
    Retorna la valoració que l'usuari 'user_id' ha donat de 'movie_id'
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb la valoració de la peli
    """
    
    # la vostra solució aquí
    return df.loc[user_id,movie_id]  # loc para buscar la fila - user_id y columna - movie_id

get_count(df_counts, 1, 1)

5.0

### 3.4. EXERCICI D

In [47]:
data.nunique()

user_id         6040
movie_id        3706
rating             5
timestamp     458455
gender             2
age                7
occupation        21
zip             3439
title           3706
genres           301
dtype: int64

In [48]:
unique_movies = pd.unique(data['movie_id'])
unique_movies.max()

3952

Si observem el nombre total d'usuaris únics i de pel.licules úniques, podem veure que els id's dels usuaris van de 1 a 6040. Normalment volem índexos que comencin al nombre 0, anant de 0 a 6039. 

+ Explora els índexos de les pel·licules. **Quin problema hi ha amb els indexos de les pel·licules??**

> **Resposta**  

> Amb el data.nunique() podem veure que el número total de pel·lícules existents és de 3706, pero el màxim ID que tenim al nostre dataFrame
> es de 3952. Tenim el problema de que hi ha indexos de les pel·licules repetits o no són consecutius, tenint informació redundant i podent causar diversos errors si no es gestionen correctament.

+ Usant la funció `pd.Categorical(*).codes`, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [49]:
# la vostra solució aquí
data['user_id'] = pd.Categorical(data['user_id']).codes # Reindexar els user_id
data['movie_id'] = pd.Categorical(data['movie_id']).codes # Reindexar els movie_id

# La funció categorical converteix la columna user_id en una categoria amb valors únics. 
# codes assigna un valor numèric a cada categoria, de manera consecutiva.


In [50]:
data[data['user_id'] == 2]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
182,2,3189,4,978298147,M,25,15,55117,Animal House (1978),Comedy
183,2,1504,2,978298430,M,25,15,55117,"Full Monty, The (1997)",Comedy
184,2,627,3,978297867,M,25,15,55117,Mission: Impossible (1996),Action|Adventure|Mystery
185,2,1295,4,978298147,M,25,15,55117,Raising Arizona (1987),Comedy
186,2,3301,3,978297068,M,25,15,55117,28 Days (2000),Comedy
187,2,101,4,978298486,M,25,15,55117,Happy Gilmore (1996),Comedy
188,2,2530,4,978297867,M,25,15,55117,"Golden Child, The (1986)",Action|Adventure|Comedy
189,2,1120,4,978297600,M,25,15,55117,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
190,2,1327,3,978297095,M,25,15,55117,Beverly Hills Ninja (1997),Action|Comedy
191,2,3622,3,978298486,M,25,15,55117,"Naked Gun: From the Files of Police Squad!, Th...",Comedy


+ Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular i visualitza ``df_counts``:

In [51]:
df_counts = build_counts_table(data)
df_counts

movie_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.5. EXERCICI E



+ Escriu una funció `distEuclid(x,y)`  que implementi la **distància** Euclidiana entre dos vectors usant funcions de numpy. 

+ Escriu la funció `simEuclid(U1, U2)` que calculi la **similitud** entre dos vectors segons la fòrmula següent (on $n$ és un factor de normalització). Ho fem així perquè si dos usuaris tenen moltes pel·licules en comú, volem que la similitud entre aquests usuaris sigui major que el de dos usuaris que només n'han vist una en comú. Si els vectors estan buits, retornar 0.

    $$d =  \frac{1}{(1+distEuclid(U1, U2))} \times \frac{len(U1)}{n} $$

+ Escriu la funció `simUsuaris(df, U1, U2)` per retorna la sembalça de dos usuaris a partir del `df_counts`, tenint en compte les puntuacions que tenen en comú, fent servir les dues funcions anteriors.
    
+ Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.* 

Per implementar aquestes funcions únicament es permet l'ús de les funcions:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**!

In [52]:
num_movies = data.nunique()['movie_id']

def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """

    dif = x - y # restar cada component del vector amb el component del altre vector
    sqrdif = np.power(dif,2)  # elevar totes les diferenciés a 2
    sqrdifsum = np.sum(sqrdif)  # sumar tots els valors

    return np.sqrt(sqrdifsum) # retornar l'arrel quadrada
    
    # la vostra solució aquí


def simEuclid(Vec1, Vec2, norm):
    """
    Retorna la sembalça de dos vectors.
    
    :param Vec1: Primer vector
    :param Vec2: Segon vector
    :return : Escalar (float) corresponent a la semblança
    """
    # la vostra solució aquí

    mask = ~np.isnan(Vec1) & ~np.isnan(Vec2) # màscara per agafar quines pelis han sigut valorades per els 2

    Vec1_both = Vec1[mask] # filtrar els elements no interesants
    Vec2_both = Vec2[mask] # filtrar els elements no interesants
    
    distEuclidiana = distEuclid(Vec1_both , Vec2_both)

    len_u1 = np.sum(mask) # sumem el nombre de pelis en comú
    
    similaritat = ( 1 / 1 + distEuclidiana ) * (len_u1/norm)
     
    return similaritat


def simUsuaris(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana
    
    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """

    U1_ratings = DataFrame.loc[User1].values  # Valoracions del primer usuari
    U2_ratings = DataFrame.loc[User2].values  # Valoracions del segon usuari
    
    similarity = simEuclid(U1_ratings, U2_ratings, num_movies) # num de movies uniques
    
    return similarity
    
    # la vostra solució aquí
    

In [53]:
print(simUsuaris(df_counts, 2,314))

0.0005396654074473826


In [54]:
# Aqui ha de srotir un valor de l'ordre de microsegons, no milisegons

%timeit simUsuaris(df_counts, 1, 5)

146 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### 3.6. EXERCICI F

En aquest exercici desenvoluparem un sistema de recomanació col·laboratiu **basat en usuaris**. 

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de similitud (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem. 

Exemple: ``getRecommendationsUser(data, 2, 50, 10, simEuclid)``

Com a sortida ha de donar la llista de les $n$ millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

#### EXERCICI F.1

+ Computa la *score* de similitud del usuari desitjat (``userID``) respecte tots els altres i retorna un diccionari dels $m$ usuaris més propers i el seu *score* de semblança. Fes servir la matriu `df_counts` Normalitzeu els *scores* de sortida de manera que sumin 1.

In [78]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : dictionary
    """
    # la vostra solució aquí

    user_vector = DataFrame.loc[userID].values  # Vector de valoracions del usuari
    other_users = DataFrame.index[DataFrame.index != userID]  # Crear una llista amb els ID dels altres usuaris únics i excloure el propi usuari

    MapUserSimilarity = map(lambda altreUser: (altreUser, simfunction(DataFrame, userID, altreUser)), other_users)
    # mapa on iterem cada usuari diferent
    # la funció del mapa és la simUsuaris + el ID del altre usuari
    # els valors són els altres usuaris


    ListUserSimilarity = list(MapUserSimilarity) # pasem a una llista

    sorted_similarities = sorted(ListUserSimilarity, key=lambda x: x[1], reverse=True) # Ordenar per el valor de similaritat en ordre descendent

    total_score = sum(score for _, score in sorted_similarities)  # Sumar les semblances
    
    top_m_users = sorted_similarities[:m]     # Seleccionar els m usuaris més similars
    
    normalized_scores = {user: score / total_score for user, score in top_m_users}  # user : valor normalitzat 
    # Crear un diccionari amb els usuaris i els seus scores normalitzats

    return normalized_scores
            
    

In [79]:
t = datetime.datetime.now()
sim_dict = find_similar_users(df_counts, 2, 10, simUsuaris)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.796653


In [80]:
sim_dict

{5794: 0.0013675975518079536,
 1180: 0.0012575820676967331,
 5366: 0.001247392852195043,
 1940: 0.0012249383371264442,
 1339: 0.0012119894387666833,
 2014: 0.0011401634360377719,
 4509: 0.0011242467744880665,
 244: 0.0010854244845921344,
 1679: 0.001075746089190595,
 4343: 0.0010703212579167795}

+ Quan trigaria (en minuts) si ho fem per tots els usuaris?

# la vostra solució aquí

> Tenint més de 6000 usuaris a la nostra matriu i trigant aproximadament 1 segon per usuari, tardariem 6000 * 1 / 60 = 10 minuts aprox.
> Encara que aquest càlcul es pot reduir ( la similiritud entre i,j és igual a la de j,i ), és un temps computacionalment car, sobretot en
> escenaris reals on no hi ha 6000 usuaris sino milions.


+ Anem ara a construir una `matriu` de mida $U \times U$ on cada posició $(i,j)$ indiqui la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3.

Compareu aquestes dues opcions des del punt de vista de temps de càlcul:

* Feu una funció,  que construeixi la ``similarity_matrix1`` a partir de la distancia entre usuaris com la distància enre els vectors formats pels elements en comú dels dos usuaris.
* Feu una funció que construeixi la ``similarity_matrix2`` d'una forma *aproximada*:
    + Substituïnt els ``nand`` que corresponen a ítems no avaluats per ``0``. 
    + Treballant específicament amb operacions matricials. En aquest link podeu trobar indicacions de com fer-ho: https://jaykmody.com/blog/distance-matrices-with-numpy/

In [123]:
def compute_similitude(fixed_arr, var_arr):
    """
    Donats dos vectors, calcula la similitud entre els subvectors formats 
    pels elements en comú (sense fer servir cap iteració!). 
    Normalitzeu la sortida multiplicant pel nombre de pel·lícules vistes en comú i
    dividint pel nombre total de pelis del dataset
    """
    
    # la vostra solució aquí
    common_ratings = ~np.isnan(fixed_arr) & ~np.isnan(var_arr) # mascara per pelis vistes pels dos

    # Obtenir les valoracions en comú
    fixed_common = fixed_arr[common_ratings]
    var_common = var_arr[common_ratings]
    
    distance = distEuclid(fixed_common, var_common)

    # Si la distància és 0 (els vectors són iguals), la similitud ha de ser 1
    if distance == 0:
        return 1

    # Calcular la similitud utilizando la distancia euclidiana
    euclidean_distance = np.linalg.norm(fixed_common - var_common)
    similarity = 1 / (1 + euclidean_distance)  # Similaridad inversa a la distancia
    
    # Normalització: multiplicar per el número de elements en comú i dividir per el total de elementos
    normalized_similarity = similarity * len(fixed_common) / num_movies#len(fixed_arr)
    
    return normalized_similarity


In [124]:
# test
vec1 = np.array([1, np.nan, 2, 3, 4])
vec2 = np.array([np.nan, 4, 5, 2, 2])
print(compute_similitude(vec1, vec2))

vec1 = np.array([1, np.nan, 1, 1, 1])
vec2 = np.array([np.nan, 1, 1, 1, 1])
print(compute_similitude(vec1, vec2))

vec1 = np.array([1, 1, 1, 1, 1])
vec2 = np.array([1, 1, 1, 1, 1])
print(compute_similitude(vec1, vec2)*num_movies/5)

0.00017072049815936371
1
741.2


In [125]:
def similarity_matrix_1(compute_distance, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """

    M = len(df_counts)  # Nombre d'usuaris (o ítems)
    similarity_matrix = np.zeros((M, M))  # Matriu de similituds inicialitzada a zeros
    
    # Iterar sobre tots els parells d'usuaris (i, j)
    for i in range(M):
        for j in range(i + 1, M):  # La matriu és simètrica, així que només cal calcular la meitat superior
            # Obtenim les valoracions de cada usuari
            fixed_arr = df_counts.iloc[i].values
            var_arr = df_counts.iloc[j].values

            total_movies = len(df_counts.columns)
            
            # Calcular la similitud entre els dos usuaris
            similarity = compute_distance(fixed_arr, var_arr)
            
            # Assignar la similitud a la matriu
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # La matriu és simètrica
    
    return similarity_matrix
    
    # la vostra solució aquí


In [ ]:
t = datetime.datetime.now()
sim = similarity_matrix_1(compute_similitude, df_counts)
t = datetime.datetime.now()-t
print("Temps amb doble for:",str(t))

In [149]:
def similarity_matrix_2(DataFrame):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    Substitueix els nand per 0.

    :return : Matriu numpy de mida M x M amb les similituds.
    """
    # la vostra solució aquí

    # Substituir NaN per 0
    DataFrame = np.nan_to_num(DataFrame, nan=0)

    # x2 -> suma de cada fila al cuadrat
    x2 = np.sum(DataFrame**2, axis=1)  # (m,) forma (m)

    # xy -> producte punt entre totes les parelles de les files de DataFrame
    xy = np.matmul(DataFrame, DataFrame.T)  # (m, m), producte de matriu (m, d) * (d, m)

    # y2 -> suma de cada columna al cuadrat 
    y2 = x2  # estem comparan la mateixa matriu, per tant x2 == y2

    # Calcular distancies Euclidianas (matriu de distancies)
    # x2 i y2 són de forma (m,1), y xy és de forma (m,m)
    x2 = x2.reshape(-1, 1)  # Convertir de (m,) a (m,1) para broadcasting
    distances = np.sqrt(x2 - 2 * xy + y2)  # (m, m)

    # Convertir distancias a similitudes (sim = 1 / (1 + distancia))
    similarities = 1 / (1 + distances)

    np.fill_diagonal(similarities, 0)

    return similarities


In [150]:
t = datetime.datetime.now()
sim = similarity_matrix_2(df_counts)
t = datetime.datetime.now()-t
print("Temps matricialment:",str(t))

Temps matricialment: 0:00:05.957546


In [151]:
sim

array([[0.        , 0.01923077, 0.02465478, ..., 0.02733667, 0.02001635,
        0.01365454],
       [0.01923077, 0.        , 0.02020307, ..., 0.02127168, 0.01646558,
        0.01340234],
       [0.02465478, 0.02020307, 0.        , ..., 0.030433  , 0.01960016,
        0.01376823],
       ...,
       [0.02733667, 0.02127168, 0.030433  , ..., 0.        , 0.02199568,
        0.01407601],
       [0.02001635, 0.01646558, 0.01960016, ..., 0.02199568, 0.        ,
        0.01347119],
       [0.01365454, 0.01340234, 0.01376823, ..., 0.01407601, 0.01347119,
        0.        ]])

+ Ara torna a re-fer la funció ``find_similar_users`` usant la matriu de distàncies i mira quant triga. Recorda que les scores han d'estar normalitzades!

In [167]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    # la vostra solució aquí
       
    # Obtenir la fila de distàncies de l'usuari seleccionat (userID)
    user_distances = sim_mx[userID]
    user_distances[userID] = 2 # evitar agafar al propi usuari
    
    suma = sum(user_distances) - 2  # sumar totes les similituts
    user_distances = user_distances / suma # dividir tot per el total de simimlituds
            
    # Obtenir els m usuaris més propers (mínimes distàncies)
    top_m_indices = np.argsort(user_distances)[:m]
    
    scores = user_distances[top_m_indices] # agafar els scores dels m usuaris més propers

    # Crear el diccionari d'usuaris similars
    similar_users = {idx: score for idx, score in zip(top_m_indices, scores)}
    
    return similar_users
    


In [168]:
t = datetime.datetime.now()
sim_dict = find_similar_users(df_counts, sim, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.007992


In [169]:
sim_dict

{4168: 4.3259093074802755e-05,
 4276: 4.4188379584851437e-05,
 1679: 4.793846250586624e-05,
 2908: 5.524311749361825e-05,
 1014: 5.568168421372354e-05,
 3031: 5.584458994346103e-05,
 1284: 5.667651445280744e-05,
 3538: 5.692068676230122e-05,
 4447: 5.736054562298388e-05,
 3390: 5.737828872032432e-05}

#### EXERCICI F.2

+ Computa les recomanacions per un usuari concret a partir dels scores dels seus $m$ usuaris més propers. 
    + Fes primer una funció ``weighted_average`` que retorni un diccionari del tipus ``{peli_id: score predit}`` amb la puntuació predita de cada ítem a partir de les puntuacions dels $m$ usuaris més propers i de la seva semblança a l'usuari considerat.
    + Fes després una funció ``getRecommendationsUser`` que usant la funció anterior retorni un ``df`` amb els $n$ ítems amb més score i els seus scores.

In [173]:
def weighted_average(DataFrame, user, sim_mx, m):
    """    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_matrix
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return: diccionari {peli_id: score predit}
    """
    # la vostra solució aquí

    # Agafar usuaris similars
    similar_users = find_similar_users(DataFrame, sim_mx, user, m)  # {id: similitud}

    id_similar_users = list(similar_users.keys())
    values_similar_users = list(similar_users.values())

    # Construir taula de valoracions
    df_counts = build_counts_table(DataFrame)

    # Pel·lícules no vistes
    viewed_movies = df_counts.loc[user][pd.notna(df_counts.loc[user])].index
    unseen_movies = df_counts.columns.difference(viewed_movies)
    

    # Valoracions i similituds per a pel·lícules no vistes
    df_counts_unseen = df_counts.loc[id_similar_users, unseen_movies]
    similarity_mat = np.array(values_similar_users).reshape(-1, 1)  # Vector column de similituds

    # Ponderació i suma
    predicted_scores = df_counts_unseen.values * similarity_mat # Calcula les puntuacions esperades ponderadas por la similitud entre usuaris
    weighted_scores = np.nansum(predicted_scores, axis=0) # Sumar les puntuacions esperades al llarg de les files
    
    similarity_sums = np.nansum(similarity_mat * ~df_counts_unseen.isnull().values, axis=0) # sumar similituds rellevants a la fila
    similarity_sums = np.where(similarity_sums == 0, 1e-6, similarity_sums) # evitar divisions entre 0

    # Prediccions 
    predicted_ratings = weighted_scores / similarity_sums  # dividir la suma dels scores entre la suma de similaritats
    predicted_dict = {movie_id: score for movie_id, score in zip(unseen_movies, predicted_ratings) if not np.isnan(score)} # crear el dict

    return predicted_dict
      


In [174]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : dataframe de pel·licules amb els scores.
    """
    
    # la vostra solució aquí

    # Obtenir les puntuacions predites per a les pel·lícules utilitzant weighted_average
    predicted_scores = weighted_average(DataFrame, user, sim_mx, m)

    # Ordenar les pel·lícules per puntuació de major a menor
    sorted_predictions = sorted(predicted_scores.items(), key=lambda x: x[1], reverse=True)

    # Ordenar el nostre dict primer en funció del rating y després en funció del ID
    # sorted_predictions = dict(sorted(predicted_scores.items(), key=lambda item: (item[1], item[0])))

    # Seleccionar les 'n' millors pel·lícules
    top_n_movies = sorted_predictions[:n]

    # Crear un DataFrame amb les pel·lícules recomanades i els seus scores
    recommended_df = pd.DataFrame(top_n_movies, columns=['MovieID', 'Score'])

    return recommended_df
    


In [175]:
t = datetime.datetime.now()
user_prediction = getRecommendationsUser(data, 3, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.794136


In [176]:
user_prediction

,MovieID,Score
0,3111,5.0
1,81,5.0
2,94,5.0
3,353,5.0
4,553,5.0
5,558,5.0
6,644,5.0
7,657,5.0
8,667,5.0
9,721,5.0


### 3.7. EXERCICI G


A continuació usarem la metrica **Mean Absolute Error (MAE)** per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari: 
+ La llista amb els scores reals d'un usuari
+ La llista amb els scores predits per aquest usuari

#### EXERCICI G.1

Anem a crear un conjunt de training i un de test de forma "ingènua":
+ Selecciona de forma aleatòria el 10% dels usuaris i guarda'ls en una llista anomenada ``test_set``.  
+ Guarda la resta en una llista anomenada ``train_set``.
+ Mira quants elements tenen aquestes llistes.

In [179]:
# la vostra solució aquí

def test_i_train_(data):

    testLen = int(len(data) * 10/100) # 10% de data
    test_set = data.sample(n=testLen)  # agafar n files aleatories

    test_index = test_set.index # agafar els indexs del test set
    train_set = data.drop(index=test_index) # construir train_set a partir dels indexs que estan a test_set

    return (test_set,train_set)

test_set,train_set = test_i_train_(data)
print(f" len test {len(test_set)} , len train {len(train_set)}")


 len test 100020 , len train 900189


+ Què passarà si calculo la matriu de similitud amb ``train_set`` i després intento predir pels usuaris de ``test_set``??

> **Resposta**

>Si calcules la matriu de similitud només amb els usuaris del train set i després intentes predir les valoracions per als usuaris del test set, el problema és que la matriu de similitud només reflecteix les relacions entre els usuaris del train set. Això vol dir que quan arribis als usuaris del test set, no tindran cap influència directa en el càlcul de les similituds, per tant, les prediccions per a aquests usuaris es basaran en les similituds amb usuaris del train set. Si els usuaris del test set són molt diferents dels del train set, les prediccions poden ser poc precises o errònies, ja que el model no ha vist mai les preferències específiques d'aquests usuaris durant el procés d'entrenament.


#### EXERCICI G.2

Cambien ara la manera de generar els conjunts per no tenir el problema anterior.

+ Seleccionarem aproximadament el 80% de les interaccions de cada usuari de ``test_set`` i les afegirem al ``train_set``. 
+ Podriem ara podem evaluar el sistema?

> Us donem el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afageixi el 80% de les intraccions al ``train_set``.

In [180]:
test_set.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
819571,4926,9,3,962660444,M,45,0,97520,GoldenEye (1995),Action|Adventure|Thriller
186637,1154,1260,4,974865025,M,25,12,90278,Sling Blade (1996),Drama|Thriller
915913,5535,21,2,959816317,M,25,16,67204,Copycat (1995),Crime|Drama|Thriller
696516,4168,2342,2,971579182,M,50,0,66048,Beyond the Poseidon Adventure (1979),Adventure
952140,5750,2515,3,958308634,F,1,0,14167,Inspector Gadget (1999),Action|Adventure|Children's|Comedy


In [181]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test 
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

user_id
0        1.2
1        1.8
2        1.2
3        0.2
4        4.4
        ... 
6035    16.6
6036     2.6
6037     0.4
6038     2.6
6039     8.8
Name: movie_id, Length: 5961, dtype: float64

Seleccionem la posició 1 i aquest ``user_id`` serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [182]:
groupby_count.reset_index().iloc[1]

user_id     1.0
movie_id    1.8
Name: 1, dtype: float64

In [183]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [184]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

TOTAL SAMPLES OF THE USER: 9
TOTAL SAMPLES OF THE USER IN TEST SET: 1


In [185]:
len(test_set_user.index)

9

In [186]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

TOTAL SAMPLES OF THE USER IN TRAIN SET: 8


In [187]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [226]:
from tqdm import tqdm

def add_testdata(traindf, test_set):
    """    
    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return: 
        - DataFrame de train que conté les dades de train juntament amb el 80% de test seleccionat
        - DataFrame de test que conté les dades de test que queden (20% restant)
    """

    # Agrupar per 'user_id' y contar les 'movie_id' de cada user
    groupby_count = test_set.groupby('user_id')['movie_id'].count().reset_index(name='movie_count')

    # Calcular el 80% de les pelis de cada user
    groupby_count['movie_count_80'] = (groupby_count['movie_count'] * 0.8).astype(int)

    # llista per agrupar el que tenim que afegir
    train_samples = []

    # llista per agrupar els indexs que tenim que eliminar
    to_remove_indices = []

    # iterar tota la taula de test agrupada amb els usuaris
    for _, row in tqdm(groupby_count.iterrows(), total=len(groupby_count), desc="Procesando usuarios"):
        
        user_id = row['user_id']  # agafar el usuari 
        n_samples = row['movie_count_80'] # agafar el valor de les pelis valorades * 0.8
        
        user_data = test_set[test_set['user_id'] == user_id] # Filtrar per usuari

        sampled_data = user_data.sample(n=n_samples) # agafar una mostra de tamany n

        train_samples.append(sampled_data)  # afegir les mostres a la llista per afegir més tard

        to_remove_indices.extend(sampled_data.index) # afegir els indexs per eliminar més tard

    traindf = pd.concat([traindf] + train_samples) # concatenar tot

    test_set = test_set.drop(index=to_remove_indices)
    # eliminar el resto

    traindf = traindf.sort_values(by=['user_id', 'movie_id']).reset_index(drop=True)
    test_set = test_set.sort_values(by=['user_id', 'movie_id']).reset_index(drop=True)
    # ordenar per user id i despres per movie
    
    return traindf, test_set


In [227]:
train, test = add_testdata(train_set, test_set)

Procesando usuarios: 100%|███████████████████████████████████████████████████████| 5961/5961 [00:04<00:00, 1202.02it/s]


In [228]:
train

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,0,0,5,978824268,F,1,10,48067,Toy Story (1995),Animation|Children's|Comedy
1,0,47,5,978824351,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance
2,0,144,5,978301777,F,1,10,48067,Apollo 13 (1995),Drama
3,0,253,4,978300760,F,1,10,48067,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
4,0,513,5,978824195,F,1,10,48067,Schindler's List (1993),Drama|War
...,...,...,...,...,...,...,...,...,...,...
977740,6039,3441,4,960971696,M,25,6,11106,Blood Simple (1984),Drama|Film-Noir
977741,6039,3461,4,964828575,M,25,6,11106,Mad Max 2 (a.k.a. The Road Warrior) (1981),Action|Sci-Fi
977742,6039,3493,4,960971654,M,25,6,11106,Serpico (1973),Crime|Drama
977743,6039,3508,4,964828782,M,25,6,11106,Chicken Run (2000),Animation|Children's|Comedy


In [229]:
test.reset_index()

,index,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,0,0,964,5,978302205,F,1,10,48067,Dumbo (1941),Animation|Children's|Musical
1,1,0,1195,5,978302039,F,1,10,48067,Ben-Hur (1959),Action|Adventure|Drama
2,2,1,1788,2,978298881,M,56,16,70072,"Breakfast Club, The (1985)",Comedy|Drama
3,3,1,3647,1,978299535,M,56,16,70072,Nurse Betty (2000),Comedy|Thriller
4,4,2,1174,5,978297396,M,25,15,55117,Unforgiven (1992),Western
...,...,...,...,...,...,...,...,...,...,...,...
22459,22459,6039,1774,3,960972782,M,25,6,11106,Rocky (1976),Action|Drama
22460,22460,6039,1794,1,956716478,M,25,6,11106,Friday the 13th (1980),Horror
22461,22461,6039,1953,1,956716294,M,25,6,11106,Weird Science (1985),Comedy
22462,22462,6039,2203,3,956704475,M,25,6,11106,Shakespeare in Love (1998),Comedy|Romance


In [230]:
train.shape

(977745, 10)

In [231]:
test.shape

(22464, 10)

In [232]:
data.shape

(1000209, 10)

In [233]:
assert train.shape[0] + test.shape[0] == data.shape[0]

#### EXERCICI G.3

+ Fes una funció que serveixi per evaluar el nostre sistema usant la mètrica MAE. 

In [234]:
from tqdm import tqdm

def evaluateRecommendations(train, test, m,n, sim):
    """
    Retorna l'error generat pel model
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar (no)
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAE
    """
   
    # la vostra solució 

    mae_total = 0
    count = 0
    
    # Itera pels usuaris en el conjunt de test amb tqdm
    for user, user_test_data in tqdm(test.groupby('user_id'), desc="Calculando MAE"):
        
        # Obtenim els scores predits per aquest usuari utilitzant el model
        predicted = getRecommendationsUser(train, user, sim, n, m)

        #print(predicted)
        
        # Obtenim les pel·lícules i els scores reals del conjunt de test per aquest usuari
        real_scores = user_test_data.set_index('movie_id')['rating']  # Índex de pel·lícules i puntuacions
        predicted_scores = predicted.set_index('MovieID')['Score']  # Índex de pel·lícules i puntuacions predits
        
        # Filtrem només les pel·lícules comunes
        common_items = real_scores.index.intersection(predicted_scores.index)
        if len(common_items) > 0:
            mae_user = abs(real_scores.loc[common_items] - predicted_scores.loc[common_items]).sum()
            mae_total += mae_user
            count += len(common_items)
    
    # Retornem el MAE global
    return mae_total / count if count > 0 else float('inf')
    


In [235]:
t = datetime.datetime.now()
mae = evaluateRecommendations(train, test, 50, 10, sim)
t = datetime.datetime.now()-t
print(str(t))

Calculando MAE: 100%|██████████████████████████████████████████████████████████████| 5961/5961 [33:21<00:00,  2.98it/s]

0:33:21.719145


In [236]:
mae

1.333333333333334

>Podem veuer com el nostre recomanador no es precís, tenint un error mitg de 1.3, sent aquest un valor molt gran per un rating de
>1-5. Posiblement per la poca quantitat de dades al dataframe 

### 3.8. EXERCICI H (exercici opcional, no obligatori)


+ **Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.

In [ ]:
# la vostra solució aquí
dataM = data[data['gender'] == 'M'].copy()
dataM.reset_index(drop=True, inplace=True)

dataM['user_id'] = pd.Categorical(dataM['user_id']).codes # Reindexar els user_id
dataM['movie_id'] = pd.Categorical(dataM['movie_id']).codes # Reindexar els movie_id



test_setM,train_setM = test_i_train_(dataM)
trainM, testM = add_testdata(train_setM, test_setM)

df_countsM = build_counts_table(dataM)
simM = similarity_matrix_2(df_countsM)

maeM = evaluateRecommendations(trainM, testM, 50, 10, simM)
maeM


Calculando MAE:   8%|█████▏                                                         | 348/4271 [01:28<21:15,  3.08it/s]

**Resposta**

>Un model per a cada sexe redueix el tamany de les nostres dades, tenint un cost computacional molt menor. A més, si asumim que els homes i les dones
>tenen un gust diferent a l'hora de escollir una pel·lícula, recomanar en funció del mateix gènere por ajudar a una recomanació més precisa. També pot pasar que si fem un recomanador mixte, i en aquest valoren més dones que homes o viceversa, podem arribar a tenir un pes major a les dones eclipsant als homes.
>
>Si la nostra suposició es falsa, agrupar les dos bases de dades seria lo millor per tenir més dades i aconseguir una precisió millor.
>
>Per comprobar la nostra teoria simplement calculem el mae dels homes i el comparem amb el mae de general. Si aquest surt menor implica que fer-ho per genere es millor. 